In [1]:
import numpy as np
import cv2
import os
import keras_ocr
import pytesseract as pt

In [2]:
# INPUT Settings
INPUT_WIDTH = 640
INPUT_HEIGHT = 640

In [3]:
# Loading the image

img = cv2.imread('./test_images/getty_sample.jpg')

cv2.namedWindow('test image',cv2.WINDOW_KEEPRATIO)
cv2.imshow('test image',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
# Load YOLO Model

net = cv2.dnn.readNetFromONNX('./Model2/weights/best.onnx')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [5]:
def get_detections(img,net):

    # Convert Image into Yolo Format

    image = img.copy()
    row,col,d = image.shape

    max_rc = max(row,col)
    input_image = np.zeros((max_rc,max_rc,3),dtype = np.uint8)
    input_image[0:row,0:col] = image

    # Get Prediction from Yolo Model

    blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WIDTH,INPUT_HEIGHT),swapRB = True,crop = False)
    net.setInput(blob)
    preds = net.forward()
    detections = preds[0]
    
    return input_image,detections

In [6]:
# Filtering the detections based on the confidence and probability value

def non_maximum_suppresion(input_image,detections):

    boxes = []

    confidences = []

    image_w,image_h = input_image.shape[:2]
    x_factor = image_w/INPUT_WIDTH
    y_factor = image_h/INPUT_HEIGHT

    for i in range(len(detections)):

        row = detections[i]
        confidence = row[4]
        if confidence>0.4:
            class_score = row[-1]
            if class_score > 0.25 :

                cx,cy,w,h = row[0:4]

                left = int((cx-0.5*w)*x_factor)
                top = int((cy-0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)
                box = np.array([left,top,width,height])

                confidences.append(confidence)
                boxes.append(box)

    # Cleaning the arrays

    boxes_np = np.array(boxes).tolist()
    confidence_np = np.array(confidences).tolist()

    # Applying Non Maximum Suppression on Bounding Box

    index = np.array(cv2.dnn.NMSBoxes(boxes_np,confidence_np,0.25,0.45)).flatten()
    
    return boxes_np,confidence_np,index

In [7]:
# Drawing the Bounding Boxes

def get_drawings(boxes_np,confidence_np,image,index):
    
    for ind in index :
    
        x,y,w,h = boxes_np[ind]
        bb_conf = confidence_np[ind]
        conf_text = 'plate : {:.0f}%'.format(bb_conf*100)
        
        license_text = OCR(image,boxes_np[ind])
        

        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.rectangle(image,(x,y-30),(x+w,y),(255,0,255),-1)
        cv2.rectangle(image,(x,y+h),(x+w,y+h+30),(0,0,0),-1)


        cv2.putText(image,conf_text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)
        cv2.putText(image,license_text,(x,y+h+27),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),1)
    
    return image


In [8]:
def yolo_predictions(img,net):
    
    # Predictions

    # Step 1 : Getting Detections

    input_image,detections = get_detections(img,net)

    # Step 2 : Applying Non Maximum Suppresion Technique

    boxes_np,confidence_np,index = non_maximum_suppresion(input_image,detections )

    # Step 3 : Gettings Drawings 

    results = get_drawings(boxes_np,confidence_np,img,index )
    
    return results

In [9]:
# Creating Pipeline for optical charecter Recognition

pipeline = keras_ocr.pipeline.Pipeline()


Looking for C:\Users\DELL\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\DELL\.keras-ocr\crnn_kurapan.h5


In [10]:
def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
        if brightness != 0:
            if brightness > 0:
                shadow = brightness
                highlight = 255
            else:
                shadow = 0
                highlight = 255 + brightness
            alpha_b = (highlight - shadow)/255
            gamma_b = shadow
            
            buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
        else:
            buf = input_img.copy()
        
        if contrast != 0:
            f = 131*(contrast + 127)/(127*(131-contrast))
            alpha_c = f
            gamma_c = 127*(1-f)
            
            buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

        return buf

In [11]:

def OCR(image,bbox):
    
    x,y,w,h = bbox
    
    roi = image[y:y+h,x:x+w]

    predictions = pipeline.recognize([roi])
    
    if predictions[0] :
        
        text = predictions[0][0][0]
        
        return text
    else :
        
        return ""


In [12]:
# Performing Image Processing 

def image_processing(img):
    
    # Read the image
    image = np.uint8(img)

    # Removing the noise from the image (Denoising)
    denoised = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

    # Sharpening
    blurred = cv2.GaussianBlur(denoised, (0, 0), 3)
    sharpened = cv2.addWeighted(denoised, 1.5, blurred, -0.5, 0)

    # Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    # Apply CLAHE separately on each channel
    enhanced_channels = [clahe.apply(channel) for channel in cv2.split(sharpened)]
    enhanced = cv2.merge(enhanced_channels)
    
    return enhanced


In [15]:
img = cv2.imread('./test_images/N50.jpeg')
# img = image_processing(img)
results = yolo_predictions(img,net)

1/1 [==============================] - 0s 186ms/step


In [16]:
cv2.namedWindow('results',cv2.WINDOW_KEEPRATIO)

cv2.imshow('results',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Real Time Object Detection with Yolo

In [17]:
cap = cv2.VideoCapture('./test_images/traffic.mp4')

while True :
    
    ret,frame = cap.read()
    
    if ret == False :
        
        print('unable to detect video')
        
    results = yolo_predictions(frame,net)
    cv2.namedWindow('results',cv2.WINDOW_KEEPRATIO)

    cv2.imshow('results',results)
    
    if cv2.waitKey(1) == 27 :
        break
        
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 120ms/step


1/1 [==============================] - 0s 115ms/step


1/1 [==============================] - 0s 83ms/step


1/1 [==============================] - 0s 173ms/step


1/1 [==============================] - 0s 194ms/step


1/1 [==============================] - 0s 154ms/step


1/1 [==============================] - 0s 158ms/step


KeyboardInterrupt: 